# Here i want to extract from lichess data features important for future modeling

So here i want to get data in format similar to that: 

columns = ( date, amount of mathes played in that date,ammont of matches played day earllier , amount of matches plyed for previous mounth, ranikng points fluctuation (day vs day before), ranking after last match that day, mean oponent vs my ranking diff that day ) and maybe some other columns that i will consider as important for ranking changes prediction. --> df2

And all of those for different times control (blitz, rapid, bullet)

and other data for winner prediction models, similar columns as without any chnages but some new cols like: time used in game, time for move and maybe some other --> df1

# To Be Continued

In [1]:
import pandas as pd
import numpy as np
import arrow
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("Chess_data_basic.csv", index_col = "Unnamed: 0")

In [4]:
df.shape

(5364, 15)

In [13]:
len(set(df["black_player_name"]+df["white_player_name"]))

4789

In [28]:
df.head(2)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6
1,KeBcqcHz,True,standard,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,resign,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",1,0,0,1,17,14,Nf3 Nc6 g3 e5


In [14]:
def convert_date_with_offset(date_str):
    return arrow.get(date_str).datetime
df['createdAt'] = df['createdAt'].apply(convert_date_with_offset)
df['lastMoveAt'] = df['lastMoveAt'].apply(convert_date_with_offset)
df["Gametime"]= df["lastMoveAt"]-df["createdAt"]
df['Gametime'] = pd.to_timedelta(df['Gametime'])
df['Gametime'] = df['Gametime'].dt.total_seconds().apply(lambda x: round(x, 1))
df['timepermove']=(df['Gametime']/df["movesperplayer"]).apply(lambda x: round(x, 1))

next think to do is taking insight from moves column, there is a lot of possibility but i will concentrate on:
1) cheking if side castle and if long or short castle
2) taking number of moves before castle ( if no castle - values set to number of moves +1 )
3) groping by opening (for this project i will take in count only first 6 moves - 3 moves per side )
4) number of checks in first 10 moves ( per side )

In [27]:
df["moves"][2]

['e4',
 'e5',
 'Nc3',
 'Nf6',
 'Nf3',
 'Nc6',
 'd4',
 'exd4',
 'Nxd4',
 'Nxd4',
 'Qxd4',
 'Qe7',
 'Bg5',
 'h6',
 'Bh4',
 'c5',
 'Qe3',
 'd5',
 'Bb5+',
 'Bd7',
 'Bxd7+',
 'Kxd7',
 'Qh3+',
 'Ke8',
 'O-O-O',
 'd4',
 'Nd5',
 'Qd6',
 'Bg3',
 'Qa6',
 'Nc7+']

In [15]:
df = df.dropna()

# Checking for side castle 

kingside castle ( short ) is O-O in moves notation and queenside castle ( long ) is O-O-O

In [16]:
pd.options.mode.chained_assignment = None

In [17]:
df["moves"] = df["moves"].str.split(" ")
df["white_moves"]=df["moves"].apply(lambda x: x[::2])
df["black_moves"]=df["moves"].apply(lambda x: x[1::2])

In [18]:
df["white_kingcastle"] = df["white_moves"].apply(lambda x: 1 if "O-O" in x else 0)
df["white_queencastle"] = df["white_moves"].apply(lambda x: 1 if "O-O-O" in x else 0)
df["black_kingcastle"] = df["black_moves"].apply(lambda x: 1 if "O-O" in x else 0)
df["black_queencastle"] = df["black_moves"].apply(lambda x: 1 if "O-O-O" in x else 0)

# num of moves before castle 

In [19]:
def find_index_of_first_occurrence(lst):
    elements = ["O-O", "O-O-O"]
    for element in elements:
        if element in lst:
            return lst.index(element)
    return len(lst) + 1
df["White_moves_before_castle"] = df["white_moves"].apply(find_index_of_first_occurrence)
df["Black_moves_before_castle"] = df["black_moves"].apply(find_index_of_first_occurrence)

In [57]:
df.head(12)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6
1,KeBcqcHz,True,standard,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,resign,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",1,0,0,1,17,14,Nf3 Nc6 g3 e5
2,hDXG2PXc,True,standard,blitz,2023-06-26 00:04:39.921000+02:00,2023-06-26 00:07:33.812000+02:00,resign,white,"[e4, e5, Nc3, Nf6, Nf3, Nc6, d4, exd4, Nxd4, N...",300,15,Gabaz,1790,EvilSaintPL,1764,173.9,11.6,"[e4, Nc3, Nf3, d4, Nxd4, Qxd4, Bg5, Bh4, Qe3, ...","[e5, Nf6, Nc6, exd4, Nxd4, Qe7, h6, c5, d5, Bd...",0,1,0,0,12,16,e4 e5 Nc3 Nf6
3,W45quoHn,True,standard,blitz,2023-06-26 00:03:49.331000+02:00,2023-06-26 00:04:36.328000+02:00,resign,black,"[e4, g6, d4, Bg7, Nf3, d6, Bf4, Bd7, Bd3, a6, ...",300,12,EvilSaintPL,1771,Harrybrave,1738,47.0,3.9,"[e4, d4, Nf3, Bf4, Bd3, O-O, c3, e5, Qd2, Qxf4...","[g6, Bg7, d6, Bd7, a6, b5, Nf6, Nh5, Nxf4, O-O...",1,0,1,0,5,9,e4 g6 d4 Bg7
4,9L3EOwxU,True,standard,blitz,2023-06-26 00:00:44.085000+02:00,2023-06-26 00:03:44.461000+02:00,resign,white,"[e4, e5, Nf3, Nc6, d4, exd4, Nxd4, Bc5, Nxc6, ...",300,19,Hakim040,1760,EvilSaintPL,1778,180.4,9.5,"[e4, Nf3, d4, Nxd4, Nxc6, Bc4, O-O, b3, exd5, ...","[e5, Nc6, exd4, Bc5, bxc6, Ne7, O-O, d5, cxd5,...",1,0,1,0,6,6,e4 e5 Nf3 Nc6
5,H0YSCwZ7,True,standard,blitz,2023-06-16 02:11:46.517000+02:00,2023-06-16 02:18:46.356000+02:00,mate,white,"[e4, e5, Nf3, Nc6, Bc4, Bc5, c3, Nf6, d4, exd4...",300,30,EvilSaintPL,1771,bureksamesom,1793,419.8,14.0,"[e4, Nf3, Bc4, c3, d4, cxd4, Bd2, Nbxd2, e5, O...","[e5, Nc6, Bc5, Nf6, exd4, Bb4+, Bxd2+, O-O, Ne...",1,0,1,0,9,7,e4 e5 Nf3 Nc6
6,N1DA9esQ,True,standard,blitz,2023-06-16 02:02:24.899000+02:00,2023-06-16 02:11:38.426000+02:00,resign,black,"[e4, e5, Nf3, Nc6, Bc4, d6, c3, Be7, d4, Bg4, ...",300,37,EvilSaintPL,1778,KancerberoMDQ,1747,553.5,15.0,"[e4, Nf3, Bc4, c3, d4, d5, Bb5+, dxc6, h3, Qd5...","[e5, Nc6, d6, Be7, Bg4, Na5, c6, Nxc6, Bh5, Nf...",1,0,0,0,12,39,e4 e5 Nf3 Nc6
7,Rfkb0XI6,True,standard,blitz,2023-06-11 02:14:11.027000+02:00,2023-06-11 02:16:56.850000+02:00,mate,white,"[e4, e5, Nf3, d6, Bc4, h6, d4, exd4, Nxd4, c5,...",300,28,EvilSaintPL,1772,edwinorlando11,1766,165.8,5.9,"[e4, Nf3, Bc4, d4, Nxd4, Qh5, Nf3, O-O, Bb5, B...","[e5, d6, h6, exd4, c5, Qf6, Nc6, Be6, a6, bxc6...",1,0,0,0,7,29,e4 e5 Nf3 d6
8,F2C4Mm1C,True,standard,blitz,2023-06-11 02:11:42.973000+02:00,2023-06-11 02:14:01.606000+02:00,resign,white,"[e4, e5, Nf3, Nc6, d4, exd4, Nxd4, Bc5, Nxc6, ...",300,20,kasper023,1860,EvilSaintPL,1777,138.6,6.9,"[e4, Nf3, d4, Nxd4, Nxc6, Bd3, Bg5, Nc3, Bh4, ...","[e5, Nc6, exd4, Bc5, bxc6, Ne7, O-O, h6, Qe8, ...",1,0,1,0,9,6,e4 e5 Nf3 Nc6
9,c5nqt4zq,True,standard,blitz,2023-06-11 02:02:10.147000+02:00,2023-06-11 02:11:27.197000+02:00,resign,black,"[d4, d5, Nf3, Nf6, e3, Bg4, h3, Bxf3, Qxf3, Nc...",300,51,elaprendizdeljaque,1735,EvilSaintPL,1771,557.0,10.9,"[d4, Nf3, e3, h3, Qxf3, Bb5, Bxc6+, Nd2, O-O, ...","[d5, Nf6, Bg4, Bxf3, Nc6, a6, bxc6, Rb8, e6, Q...",1,0,1,0,8,13,d4 d5 Nf3 Nf6


Grouping by opening

In [21]:
df["open6"] = df["moves"].apply(lambda x: x[:4])
df["open6"] = df["open6"].apply(lambda x: " ".join(x))

In [33]:
white_opens = df[df['white_player_name'] == 'EvilSaintPL']["open6"].value_counts().head(20)
black_opens = df[df['black_player_name'] == 'EvilSaintPL']["open6"].value_counts().head(20)

In [78]:
result = list()
def openings2( df ):
    for index, row in df.iterrows():
        if df["white_player_name"][index]=="EvilSaintPL" and df["open6"][index] in white_opens.index:
            result.append("White " + df["open6"][index])
        elif  df["white_player_name"][index]!="EvilSaintPL" and df["open6"][index] in black_opens.index:
            result.append("Black " + df["open6"][index]) 
        else:
            result.append("no opens")
    df["opens1"]=result
    return df

In [79]:
df = openings2( df )

In [82]:
df.drop(labels = ["no opens", "opens1"], axis=1, inplace=True)

In [80]:
temp = pd.get_dummies(df["opens1"])
df = pd.concat([df,temp], axis=1)


In [83]:
df.head(2)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open6,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nc6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,KeBcqcHz,True,standard,blitz,2023-06-28 02:31:45.081000+02:00,2023-06-28 02:33:30.178000+02:00,resign,white,"[Nf3, Nc6, g3, e5, Bg2, e4, Ng1, d5, d3, f5, d...",300,18,convict-19,1801,EvilSaintPL,1758,105.1,5.8,"[Nf3, g3, Bg2, Ng1, d3, dxe4, Nc3, Bg5, Bxf6, ...","[Nc6, e5, e4, d5, f5, fxe4, Nf6, Be7, Bxf6, Qx...",1,0,0,1,17,14,Nf3 Nc6 g3 e5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
